# FGV:  MBA - Business Analytics e Big Data

 

In [1]:
# Importa libs básicas e carrega arquivo do github. 
import pandas as pd
import re

df_news = pd.read_json('https://raw.githubusercontent.com/wunderbarpeter/mba_data/main/Sarcasm_Headlines_Dataset_v2.json', lines = True)

print("Dataframe original:")
print("-- -- -- "*5)
df_news.head(5)

Dataframe original:
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 


,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


Como se pode esperar, a classificação inicial e supervisionada das notícias entre sarcásticas e não-sarcásticas está relacionada às suas fontes. As manchetes do HuffPost são consideradas não-sarcásticas; as do The Onion sarcásticas. Não existem exceções. 

A distribuição abaixo demonstra o balanço entre as duas classes de manchetes na base de dados. 

In [2]:
# Mostra frequencias básicas:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

tallies = df_news['is_sarcastic'].value_counts()
label = ['Não Sarcásticas', 'Sarcásticas']
rates = (np.array((tallies/tallies.sum())*100))

px.pie(tallies, label, rates, title="Distribuição das Manchetes").\
        update_layout(title={'x': 0.45}, 
                      paper_bgcolor='mintcream',
                      width=800,
                      height=600)

# Limpeza e tratamento de dados

A limpeza e preparação dos dados comportou três etapas:
*   A remoção de caracteres especiais e pontuações, irrelevantes ao processamento de texto. 
*   A limpeza das "stopwords", como artigos, preposições, conjunções e pronomes.
*   O processamento do texto por "Stemming" e "Lemmatization"


## Tratamento Inicial
A limpeza básica foi realizada com expressões regulares e constantes de pontuação do Python `string.punctuation`. Em conjunto, utilizamos os dicionários da biblioteca `nltk` para tratar as `stopwords` e remover das manchetes. 
O resultado é uma nova coluna no dataframe `df_news` onde podemos comparar a manchete original `'headline'` com a limpa `'headline_clean'`.


In [3]:
# Limpeza de dados/texto:
import re # Regex 
import nltk # Importação de dicionários
import string # Dicionário de constantes de texto
nltk.download('stopwords', quiet=True)
stopwords = nltk.corpus.stopwords.words('english')

# Limpa dados de execução anterior:
if 'headline_clean' in df_news.columns:
  del df_news['headline_clean']

display(
    print("Verifica nulos"),
    df_news.isna().sum(), # Conferindo nulos.
    print("-- -- --"*4), 
)

# Remove outlier erro de classificação:
df_news = df_news.drop(index=7302, axis = 0) #5x mais palavras que a média
df_news.reset_index(drop=True, inplace=True)

# Remove coluna de link.
if 'article_link' in df_news.columns:
  del df_news['article_link'] 

# Define função para remover stopwords
def remove_stopwords(text): 
  non_stop = []
  for i in text.split():
    if i.strip().lower() not in stopwords:
      non_stop.append(i.strip())
  return " ".join(non_stop)

# Define função com expressões regulares para limpeza:
def cleantext(text):
  c_text = text.translate(str.maketrans('', '', string.punctuation))
  c_text = remove_stopwords(c_text)
  return c_text

Verifica nulos
-- -- ---- -- ---- -- ---- -- --


None

is_sarcastic    0
headline        0
article_link    0
dtype: int64

None

In [4]:
# Cria nova coluna com dados limpos
df_news['headline_clean'] = df_news['headline'].apply(cleantext)
print("Manchetes após a limpeza e remoção de 'stowords'")
print("-- "*20)
df_news.loc[:, ['headline', 'headline_clean']].head(10)

Manchetes após a limpeza e remoção de 'stowords'
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 


,headline,headline_clean
0,thirtysomething scientists unveil doomsday clo...,thirtysomething scientists unveil doomsday clo...
1,dem rep. totally nails why congress is falling...,dem rep totally nails congress falling short g...
2,eat your veggies: 9 deliciously different recipes,eat veggies 9 deliciously different recipes
3,inclement weather prevents liar from getting t...,inclement weather prevents liar getting work
4,mother comes pretty close to using word 'strea...,mother comes pretty close using word streaming...
5,my white inheritance,white inheritance
6,5 ways to file your taxes with less stress,5 ways file taxes less stress
7,richard branson's global-warming donation near...,richard bransons globalwarming donation nearly...
8,shadow government getting too large to meet in...,shadow government getting large meet marriott ...
9,lots of parents know this scenario,lots parents know scenario


## Distribuição de Frequências

Com a limpeza básica dos textos já é possível ter uma visão mais clara do conteúdo e sentido das palavras, de acordo com o perfil das manchetes. 
Abaixo visualizamos uma simples distrbuição de frequências da quantidade de palavras usadas de acordo com o tipo de manchete. 

In [5]:
# Volume de palavras por tipo de manchete:
df_news['word_count'] = df_news['headline_clean'].str.split().map(lambda x: len(x))

s_sarc = df_news['word_count'][df_news.is_sarcastic==1]
s_not_sarc = df_news['word_count'][df_news.is_sarcastic==0]

As notícias classficadas como sarcásticas tendem, em média, a serem mais sucintas, mas com uma amplitude maior na distribuição. Isso indica que as notícias não sarcásticas seguem um padrão mais regular de composição do texto, no que se refere ao tamanho da manchete.

In [6]:
# Compara distribuições de acordo com categoria:
hist = go.Figure()
hist.add_trace(go.Histogram(x=s_not_sarc, name="Não Sarcástico"))
hist.add_trace(go.Histogram(x=s_sarc, name="Sarcásticos"))
hist.update_layout(barmode='overlay', 
                   title = "Distribuição Contagem de Palavras",
                   width = 800, 
                   height = 800,
                   legend=dict(
                       yanchor="top", 
                       y=0.99, 
                       xanchor="left",
                       x=0.78
                   )
)
hist.update_xaxes(title="Qtd. palavras")
hist.update_yaxes(title="Frequência")
hist.update_traces(opacity=0.55)
hist.show()

## Stemming e Lemmatization
Na ausência de expressões equivalentes na língua Portuguesa, os processos de 'stemming' e 'lemmatization' vizam a redução das palavras, eliminando suas formas flexivas e/ou derivadas, mantendo seus sentidos e raízes morfológicas.

Enquanto o método 'Stemming' procura reduzir a palavra ao seu radical mais básico, o método 'Lemmatization' se utiliza de vocabulário e análise morfológica das palavras.

Podemos observar essas diferenças abaixo, na comparação entre as palavras `playing` e `wolves` nos exemplos com as classes `SnowballStemmer` e `WordNetLemmatizer`. 

In [7]:
# Stemming:
import nltk
nltk.download('wordnet', quiet=True)
from nltk.stem import WordNetLemmatizer
stemmer = nltk.stem.SnowballStemmer("english")
wlm = nltk.wordnet.WordNetLemmatizer()

print("-- --"*5)
print("Snowball Stemmer:")
print("The stemmed form of 'playing' is: {}".format(stemmer.stem("playing")))
print("The stemmed form of 'played' is: {}".format(stemmer.stem("played")))
print("The stemmed for of 'wolves' is: {}".format(stemmer.stem("wolves")))
print("-- "*6)
print("Word Lemmatizer:")
print("The lemmatized form of 'playing': {}".format(wlm.lemmatize("playing")))
print("The lemmatized form of 'wolves': {}".format(wlm.lemmatize("wolves")))
print("The lemmatized form of 'leaves': {}".format(wlm.lemmatize("leaves")))
print("-- --"*5)


-- ---- ---- ---- ---- --
Snowball Stemmer:
The stemmed form of 'playing' is: play
The stemmed form of 'played' is: play
The stemmed for of 'wolves' is: wolv
-- -- -- -- -- -- 
Word Lemmatizer:
The lemmatized form of 'playing': playing
The lemmatized form of 'wolves': wolf
The lemmatized form of 'leaves': leaf
-- ---- ---- ---- ---- --


O método escolhido para a simplificação do vocabulário do modelo foi o de 'Lemmatization', processando as listas com o `WordNetLemmatizer`. Esse tratamento foi necessário para a composição dos Bigramas e Trigramas, conforme veremos adiante. 


# Análise de Contexto

Para que o processo de vetorização e frequência de termos no texto levasse em consideração, também, o contexto, foram incorporados na matriz TF-IDF '*bigramas*' e '*trigramas*' de palavras. Em síntese, a frequência de combinações de uma, duas ou três palavras em sequência.

Essencialmente, buscamos diferenciar contextos em que palavras combinadas produzam um sentido distinto do que se analizadas individualmente. Por exemplo:
*   *Climate* (clima)
*   *Climate change* (mudanças climáticas)
*   *Climate change denial* (negacionismo climático)

Conforme apontado acima, submetemos as listas de palavras a um processo de '*Lemmatization*' para simplificar o vocabulário, mantendo seu sentido e morfologia. 

In [8]:
# Criação de listas para N-Gramas:
news = []
for entry in range(0, df_news.shape[0]):
  head_txt = df_news.headline_clean[entry]
  head_txt = head_txt.split(" ")
  news.append(head_txt)


# Criação de lista "lemmatized":
words_lemm = []
for batch in news:
    list_words = [word for word in batch if word.lower() not in stopwords]
    lemm = WordNetLemmatizer()
    list_lemm =  [lemm.lemmatize(word) for word in list_words]
    words_lemm.append(list_lemm)


Abaixo são definidas as funções para o tratamento e quebra do texto em `'n_grams'`. Os valores de entrada precisam ser passados como listas para que a função as processe corretamente. Essa estrutura é gerada no bloco anterior de código e passadas para a camada apresentada abaixo.

In [9]:
# Define funções para criação dos N-Gramas:
from collections import defaultdict
df_words_lemm = pd.DataFrame({'words' : words_lemm})

# Função para criação dos n-gramas:
"""
# Parâmetros de entrada são:
1. text = estrutura tipo 'list' com as palavras.
2. n_gram = número de palavras para combinações.
"""
def generate_ngrams(text, n_gram=1):
  ngrams = zip(*[text[i:] for i in range(n_gram)])
  return [" ".join(ngram) for ngram in ngrams]

# Função para criação do dicionário de frequências:
""" 
Parâmetros são:
1. data = dataframe com listas de palavras. 
2. n_gram = número de combinações. 
"""

def list_ngrams (data, n_gram=1):
  freq_dict = defaultdict(int)
  for sent in data:
    for word in generate_ngrams(sent,n_gram):
      freq_dict[word] += 1
  result_df = pd.DataFrame.from_dict(
      freq_dict, orient='index').\
      sort_values(by=0, ascending=False)
  result_df.reset_index(level=0, inplace=True)
  result_df.columns = ['combination', 'frequency']
  return (result_df)

Após a definição das funções, podemos criar os dois conjuntos de 'duas-palavras' e 'três-palavras' para comparar suas frequências. Aqui, não separamos a análise por categorias. Analisamos o texto como um todo, sem distinguir as manchetes "sarcásticas" e "não-sarcásticas".

## Frequências dos N-Gramas:
É possível observar que nas frequências mais altas não existe uma diferença substancial de significado entre as combinações de duas e três palavras. 

Os ranks com as 20 combinações mais frequentes apresetam ocorrências similares. 

In [10]:
# Cria datasets para os gráficos:
bigrams = list_ngrams(df_words_lemm['words'], 2)
trigrams = list_ngrams(df_words_lemm['words'], 3)

# Plota Frequências Bigramas:
import plotly.express as px

px.bar(bigrams.head(20).sort_values(by='frequency', ascending=True), 
       x='frequency', y='combination', 
       color='frequency', 
       orientation='h', width=1000, title="Frequências Bigramas",
       color_continuous_scale=px.colors.sequential.Viridis,
       labels=dict(frequency="Frequência", combination="Combinação"))

Naturalmente, as frequências para os 'trigramas' são menores, mas os contextos similares.

In [11]:
# Frequências Trigramas:
px.bar(trigrams.head(20).sort_values(by='frequency', ascending=True), 
       x='frequency', y='combination', 
       color='frequency', 
       orientation='h', width=1000, title="Frequências Trigramas",
       color_continuous_scale='Inferno',
       labels=dict(frequency="Frequência", combination="Combinação"))

# Criação dos Modelos
O modelo preditivo criado utilizou a matriz de frequência de termos com pesos e foi treinado em uma rede neural com camadas densas. 
Como já descrito acima, alguns critérios foram estabelecidos para a composição das matrizes esparsas. 

Nas seções seguintes descrevemos em detalhe essas etapas.

## Criação da Matriz Esparsa
Além das combinações de N-gramas em conjuntos de 'uma', 'duas' ou 'três' palavras, o processo de tokenização e vetorização utilizou também critérios de pesos.
Utilizamos o `TfidfVectorizer` para conversão em TF-IDF no lugar do 
`CountVectorizer`. O objetivo é fazer uma atribuição de pesos às palavras e/ou *n-grama*, ao invés de uma mera atribuição binária de frequências. O TfidfVectorizer atribui pesos de acordo com a frequência das palavras no conjunto de texto, com o intuito de captar melhor o contexto. 
Na matriz esparsa resultante, quanto mais frequente a palavra ou *n-grama*, menor seu peso no modelo.

In [12]:
# Cria a matriz esparsa com vetorização para treino do modelo. 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif

"""
O TfidfVectorizer atribui pesos de acordo com a frequência das palavras no
conjunto de texto, com o intuito de captar melhor o contexto. 
Quanto mais frequente a palavra, menor seu peso no modelo.

"""
# Instancia a vetorização com uni, bi e trigramas:
vectorizer = TfidfVectorizer(
    min_df=3, binary=True, 
    analyzer='word', ngram_range=(1,3), 
    stop_words='english')

unigram_vector = TfidfVectorizer(
    min_df=3, binary=True, 
    analyzer='word', ngram_range=(1,1), 
    stop_words='english')

# Cria as matrizes esparsas:
wm_bitrigram = vectorizer.fit_transform(df_news['headline'])
# Apenas para comparação
wm_unigram = unigram_vector.fit_transform(df_news['headline'])



Ao comparar os conjuntos de palavras distintas com o conjunto de palavras mais as combinações, pode-se perceber o incremento de elementos no modelo.

Em seguida, submetemos o objeto de vetorização da matriz esparsa ao conjunto de dados para a transformação dos termos em pesos. Para isso usamos o método `SelectKBest` da biblioteca de `feature_selection`do `sklearn`. 

O produto do processo é uma matriz esparsa comprimida com 15.014 colunas e mais de 28.000 linhas.

In [13]:
# Tamanho dos conjuntos:
print("Número de palavras distintas: {}".format(
    len(unigram_vector.get_feature_names()))
)
print("Número de Tokens (combinações): {}".format(
    len(vectorizer.get_feature_names()))
)

Número de palavras distintas: 10685
Número de Tokens (combinações): 15014


In [14]:
# Seleciona número de features a serem incluídas no treinamento:
k=len(vectorizer.get_feature_names())

selector = SelectKBest(chi2, k=min(k, wm_bitrigram.shape[1]))
selector.fit(wm_bitrigram, df_news.is_sarcastic)
transformed_texts = selector.transform(wm_bitrigram).astype('float32')

# Tamanho da matriz esparsa:
print("Matriz esparsa aponta cerca de {}".\
      format(round(
          transformed_texts.count_nonzero(),
          -3)
      ),
      "elementos.")

Matriz esparsa aponta cerca de 203000 elementos.


## Divisão dos dados
A seleção do modelo não conta com nenhuma peculiaridade. Divimos os conjuntos de dados de treinamento e validação em 80% e 20% da base, respectivamente. 
Como o conjunto de dados é pequeno para o número de *features* no modelo, procuramos trabalhar com o maior volume possível de dados. 

In [15]:
# Divide conjuntos de treinamento e teste:
from sklearn.model_selection import train_test_split

y_values = df_news['is_sarcastic'].values
x_train, x_test, y_train, y_test = train_test_split(transformed_texts, y_values, test_size=0.2)

## Contrução da Rede Neural
 
A rede neural escolhida foi a de camadas densas, em contra partida a uma rede convolutiva ou de Pooling. Como a saída ou predição é do tipo categórica, ativação final foi do tipo `sigmoid`. 
 
Na construção, ou identificação do melhor modelo procuramos testar diversas combinações de:
*   Camadas
*   Número de Neurons
*   Dropouts
 
Para tal interação, uma função básica de construção e teste do modelo foi criada (vide código abaixo para função `create_model`), os resultados de acurácia são armazenados em um dicionário de históricos do processo de *fitting* do modelo (`model_hist`) e outro dicionário com o armazenamento da combinação de parâmetros e resultado da acurácia na validação (`model_accur`). 
 
Apenas o número de épocas foi fixado em **15**, para evitar que o processo de testes com os modelos se tornasse muito extenso.

In [16]:
# Construção da Rede Neural
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras import optimizers

# Máximo de features precisa ser determinado apriori.
max_features=min(k, wm_bitrigram.shape[1])

if 'model_hist' not in locals():
  model_hist = {} # Inicializa dicionário de históricos.
if 'max_accur' not in locals():
  max_accur = {} # Inicializa dicionario de acurácias
if 'max_val_accur' not in locals():
  max_val_accur = {}

# Função de construção do modelo:
def create_model(neurons, epochs, layers=1, dropout=0, 
                 plot_model=False, # cria gráfico de resultados
                 store_hist=False # armazena histórico da validação do modelo.
                 ):
  model = Sequential()
  model.add(Dense(neurons, input_dim=max_features, activation='relu'))
  if dropout != 0: # Droput é um argumento opcional.
    model.add(Dropout(dropout))
  if layers != 1:
    if layers > 1:
      for i in range(1, layers):
        model.add(Dense(neurons, activation='relu'))
        if dropout > 0:
          model.add(Dropout(dropout))
    elif layers < 1:
      return(print("Invalid number of hidden layers"))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(optimizer='rmsprop', loss='binary_crossentropy',
                metrics=['accuracy'])
  
  m_hist = model.fit(x_train,
                     y_train,
                     batch_size = 1024,
                     epochs=epochs,
                     verbose=False,
                     validation_data = (x_test, y_test)
                     ).history

  # Plota resultados.
  if plot_model:
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=pd.DataFrame(m_hist).index,
                             y=m_hist['accuracy'],
                             mode = 'lines', 
                             name="Treino"))
    fig.add_trace(go.Scatter(y=m_hist['val_accuracy'],
                             mode='lines', 
                             name="Validação"))
    fig.add_trace(go.Scatter(y=m_hist['loss'],
                             mode='lines',
                             name='Loss'))
    fig.update_layout(title=str("Comparação Acurácia Modelo ")+str(neurons),
                      yaxis = dict(tick0=0.5, dtick=0.05),
                      width = 800,
                      height = 800,
                      )
    fig.show()
  # Salva resultados do modelo no dicionário.
  if store_hist:
    model_hist[str(neurons)+str('n_')+
               str(layers)+str('l_')+
               str(dropout)+str('d')] = m_hist
    # Garbage collection.
    del m_hist
    del model
    return (model_hist.keys())
  else:
    # Armazena Score no Dicionário de resultados
    max_accur[str(neurons)+str('n_')+
               str(layers)+str('l_')+
               str(dropout)+str('d')] = max(m_hist['accuracy'])
    
    max_val_accur[str(neurons)+str('n_')+
                  str(layers)+str('l_')+
                  str(dropout)+str('d')] = max(m_hist['val_accuracy'])
    # Garbage collection.
    del model
    del m_hist
    return None
  

### Determinação de hiperparâmetros:
Para automatizar o processo de ajuste, testamos diversos parâmetros em combinação e armazenamos o resultado em um indicador. 
O indicador calcula a composição entre:
1.   A melhor acurácia de validação e
2.   a menor distância entre a acurácia de treino e a de validação.


> *A execução do processo abaixo pode levar entre 20 e 40min.*



In [ ]:
# Sequencias de hiper-parâmetros:
l_neurons = list(np.arange(2, 21, 1))
l_layers = [1, 2, 3]
l_dropout = [0.0, 0.2, 0.4, 0.5]
 
# Automatiza os testes com modelos e vários parâmetros:
for n in l_neurons:
  for l in l_layers:
    for d in l_dropout:
      create_model(
          neurons=n,
          epochs=15,
          layers=l,
          dropout=d
          )

In [18]:
# Parametros do melhor modelo:
print("Modelo com melhor acurácia de treino: {}".format(
    max(max_accur, key=max_accur.get)
    )
)

print("Modelo com melhor acurácia na validação: {}".format(
    max(max_val_accur, key=max_val_accur.get)
    )
)


Modelo com melhor acurácia de treino: 19n_3l_0.0d
Modelo com melhor acurácia na validação: 8n_3l_0.0d


### Modelo identificado

No exame de hiperparametros o melhor modelo identificado foi obtido pela seguinte composição:
*   8 Neurons.
*   3 Camadas densas.
*   Sem Dropout entre camadas.
*   15 épocas de treino.

A acurácia obtida foi aproximadamente **80%**, ainda com um grande *overfit* durante o treinamento. 

O escrutínio da performance do modelo pode ser invocado usando os parametros de projeção no gráfico e armazenamento do histórico, conforme visualizamos abaixo. 

In [ ]:
# Roda modelo individualmente:
create_model(
    neurons=8,
    epochs=15,
    layers=3, 
    dropout=0,
    plot_model=True,
    store_hist=True)


# Teste com novos dados
O modelo e dados testados apresentaram dois aspectos problemáticos. Primeiro, a quantidade de *features* identificadas foi muito grande, quando comparada com o tamanho da base de dados. 
Segundo, os dados em si possuem um vício de origem. As manchetes categorizaddas como sarcásticas e não sarcásticas podem ser inteiramente separadas usando as fontes das matérias: The Onion e HuffPost. 

Sugerem que a performance do modelo contra dados novos, que não se enquadrem no vocabulário e perfis de escrita nem do The Onion, nem no HuffPost podem ser difíceis de se categorizar. 

Para testar essa suposição, criou-se um novo conjunto de dados para validação. Os novos dados possuem quatro fontes distintas:

*   Novas manchetes "sarcásticas" do The Onion. 
*   Manchetes "não sarcásticas" do jornal CNN.
*   Um conjunto de sentenças ou ditados "sarcásticos" populares sem origem determinada. 
*   Notícias "sarcásticas" inventadas pelos autores deste trabalho. 

Abaixo visualiza-se a carga do novo conjunto de dados. 



In [23]:
# Novos dados para teste:
newData = {'headline' : ['Last night Atlanta Hawks did a excellent job losing for Chicago Bulls',
                         'New delta strain causes peak in Covid-19 cases',
                         'I feel so miserable without you, it’s almost like having you here',
                         'Sorry for being late. I got caught up enjoying my last few minutes of not being here',
                         'There’s someone for everyone, and the person for you is a psychiatrist',
                         'I clapped because it’s finished, not because i liked it',
                         'I’m not saying I hate you. But, I would unplug you’re life support to charge my phone',
                         'The latest on Afghanistan as US troop withdrawal deadline looms',
                         'McDonald has run out of milkshakes in the UK',
                         'Anti-mask New York Post requires all of its employees to wear masks in latest sign of Murdoch media hypocrisy',
                         'Amazon just scored a major win in the battle for India retail market',
                         'Loneliness Most Common Amongst Americans No One Wants To Be Around',
                         'CDC Warns Going Unvaccinated Not Worth Risk Of Losing Ability To Taste Wings',
                         'OnlyFans CEO Admits Decision To Ban Pornography Was Made In Shame-Filled Moment After Orgasm',
                         'Nervous Biden Rushes Past Intimidating Circle Of Senators Smoking Weed On Capitol Steps',
                         'Man Mid-Shower Facing Grim Realization He’ll Have To Retrieve Face Wash He Left On Sink'],
           'is_sarcastic' : [1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1],
           'source' : ['custom', 'custom', 'custom', 'custom', 'custom', 'custom', 'custom', 
                       'CNN', 'CNN', 'CNN', 'CNN', 
                       'TheOnion', 'TheOnion', 'TheOnion', 'TheOnion', 'TheOnion']}

print(pd.DataFrame(newData))

                                             headline  is_sarcastic    source
0   Last night Atlanta Hawks did a excellent job l...             1    custom
1      New delta strain causes peak in Covid-19 cases             0    custom
2   I feel so miserable without you, it’s almost l...             1    custom
3   Sorry for being late. I got caught up enjoying...             1    custom
4   There’s someone for everyone, and the person f...             1    custom
5   I clapped because it’s finished, not because i...             1    custom
6   I’m not saying I hate you. But, I would unplug...             1    custom
7   The latest on Afghanistan as US troop withdraw...             0       CNN
8        McDonald has run out of milkshakes in the UK             0       CNN
9   Anti-mask New York Post requires all of its em...             0       CNN
10  Amazon just scored a major win in the battle f...             0       CNN
11  Loneliness Most Common Amongst Americans No On...           

Criamos um novo modelo, agora com os dados de toda a base como treinamento e usando os hiperparâmetros previamente identificados. 

In [24]:
# Inicializa a Tokenização e Vetorização dos dados novos. 
X = vectorizer.transform(newData['headline'])
Y = newData['is_sarcastic']

In [26]:
# Cria novo modelo para nova validação:
model_new = Sequential()
model_new.add(Dense(8, input_dim=max_features, activation='relu'))
model_new.add(Dense(8, activation='relu'))
model_new.add(Dense(8, activation='relu'))
model_new.add(Dense(1, activation='sigmoid'))
model_new.compile(optimizer='rmsprop', loss='binary_crossentropy',
                  metrics=['accuracy'])
# Treina modelo com toda a base:
model_new.fit(transformed_texts,
               y_values,
               batch_size = 1024,
               epochs=15,
               verbose=False)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning:

Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_233/dense_703/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_233/dense_703/embedding_lookup_sparse/Reshape:0", shape=(None, 8), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_233/dense_703/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.



É possível visualizar nas comparações de classificação com previsões do modelo uma tendência a maior precisão às manchetes do The Onion e as manchetes não-sarcásticas de veículos profissionais de notícias.
Entretanto, quando as fontes são de outro perfil, como as frases populares ou notícias inventadas as taxas de acerto variam. 

In [27]:
# Executa as classificações baseadas no novo modelo. 
predictions =  model_new.predict(vectorizer.transform(newData['headline']).sorted_indices())
df_results = pd.DataFrame(newData)
df_results['pred_score'] = predictions
df_results['pred_class'] = np.where(df_results['pred_score']>0.50, 1, 0)
df_results

,headline,is_sarcastic,source,pred_score,pred_class
0,Last night Atlanta Hawks did a excellent job l...,1,custom,0.068161,0
1,New delta strain causes peak in Covid-19 cases,0,custom,0.231084,0
2,"I feel so miserable without you, it’s almost l...",1,custom,0.646425,1
3,Sorry for being late. I got caught up enjoying...,1,custom,0.879568,1
4,"There’s someone for everyone, and the person f...",1,custom,0.860961,1
5,"I clapped because it’s finished, not because i...",1,custom,0.988875,1
6,"I’m not saying I hate you. But, I would unplug...",1,custom,0.072709,0
7,The latest on Afghanistan as US troop withdraw...,0,CNN,0.146260,0
8,McDonald has run out of milkshakes in the UK,0,CNN,0.007349,0
9,Anti-mask New York Post requires all of its em...,0,CNN,0.410350,0


# Conclusões:
Considerando os desafios técnicos do modelo e os desafios semânticos do uso de sarcasmo em comunicação, o modelo treinado com redes neurais apresentou resultados relativamente satisfatórios. Conforme podemos confirmar abaixo, a avaliação da acurácia com dados novos permaneceu em torno de **80%**. Resultado similar a fase de validação do modelo.

Identificamos ainda espaço para melhorias e ajustes na modelagem para se reduzir, sobretudo, o *overfitting* do treino, que permaneceu em tono de 10% conforme as épocas de treinamento aumentavam. Outras possibilidades são a redução do número de *features*, optando por favorecer os conjuntos de bi ou trigramas, no lugar de palavras isoladas. 

In [28]:
# Avalia a acurácia contra os dados novos. 
pd.DataFrame.from_dict(
    model_new.evaluate(x=vectorizer.transform(newData['headline']).sorted_indices(),
                       y=pd.DataFrame(newData)['is_sarcastic'].values,
                       verbose=0,
                       return_dict=True),
                       orient='index',
                       columns=['Results']
)

,Results
loss,0.529867
accuracy,0.812500
